LangChain Expression Language (LCEL) is a declarative approach to building Runnables, where you describe what should happen rather than how it should happen, allowing LangChain to optimise run-time execution. A Runnable created using LCEL is often referred to as a "chain" and it implements the full Runnable Interface.

Here’s a summary of LCEL's key features:

*   **Optimised Execution:** LCEL optimises the execution of chains through parallel processing and asynchronous support.
    *   It allows for **parallel execution**, which can reduce latency by processing tasks concurrently.
    *   It provides **guaranteed asynchronous support**, useful for handling multiple requests concurrently in a server environment.
    *   It simplifies **streaming**, optimising output to minimise time-to-first-token.

*   **Additional Benefits**:
    *   LCEL provides seamless integration with **LangSmith tracing**, logging all steps automatically for better observability and debugging.
    *   Chains built with LCEL use a **standard API**, allowing them to be used like any other Runnable.
    *   These chains can be **deployed using LangServe** for production use.

*   **When to Use LCEL:**
    *   LCEL is suitable for simpler orchestration tasks, such as simple chains involving a prompt, LLM, and parser or simple retrieval setups.
    *   If you are making a single LLM call, you don't need LCEL; instead call the underlying chat model directly.
    *   For complex applications with state management, branching, cycles, or multiple agents, it's recommended to use **LangGraph** instead. However, you can use LCEL within individual nodes in LangGraph.

*   **Composition Primitives:** LCEL chains are built by composing existing Runnables, with the main composition primitives being **RunnableSequence** and **RunnableParallel**.
    *   **RunnableSequence** chains runnables sequentially, with the output of one serving as the input to the next. The following code:
        ```
        from langchain_core.runnables import RunnableSequence
        chain = RunnableSequence([runnable1, runnable2])
        final_output = chain.invoke(some_input)
        ```
        is equivalent to:
        ```
        output1 = runnable1.invoke(some_input)
        final_output = runnable2.invoke(output1)
        ```
    *   **RunnableParallel** runs runnables concurrently, providing the same input to each. The following code:
    ```
    from langchain_core.runnables import RunnableParallel
    chain = RunnableParallel({
        "key1": runnable1,
        "key2": runnable2,
    })
    final_output = chain.invoke(some_input)
    ```
    will yield:
    ```
    {
        "key1": runnable1.invoke(some_input),
        "key2": runnable2.invoke(some_input),
    }
    ```
    The runnables are executed in parallel, resulting in faster execution time. RunnableParallel supports both synchronous (using ThreadPoolExecutor) and asynchronous (using asyncio.gather) execution.
*   **Composition Syntax:**
    *   The `|` operator is overloaded to create a **RunnableSequence**.
        *   `chain = runnable1 | runnable2` is equivalent to `chain = RunnableSequence([runnable1, runnable2])`.
    *   The `.pipe` method can be used as an alternative to the `|` operator.
        *   `chain = runnable1.pipe(runnable2)` is equivalent to `chain = runnable1 | runnable2`.
    *   **Automatic Type Coercion:**
        *   A dictionary is automatically converted to a **RunnableParallel** within an LCEL expression. For example:
          ```
          mapping = {
              "key1": runnable1,
              "key2": runnable2,
          }
          chain = mapping | runnable3
          ```
          is converted to:
          ```
          chain = RunnableSequence([RunnableParallel(mapping), runnable3])
          ```
        *  A function is automatically converted to a **RunnableLambda** within an LCEL expression. For example:
           ```
           def some_func(x):
               return x
           chain = some_func | runnable1
           ```
           is converted to:
           ```
           chain = RunnableSequence([RunnableLambda(some_func), runnable1])
           ```
*   **Legacy Chains:** LCEL aims to provide consistency and customisation, unlike legacy chains such as `LLMChain` and `ConversationalRetrievalChain`. It's recommended to migrate from these legacy chains to LCEL for better customisation and clarity.

In summary, LCEL offers a powerful way to build and optimise LangChain applications through its declarative approach, composition primitives, and shorthand syntax. It also provides enhanced performance through parallel processing, asynchronous support, and seamless streaming capabilities. It is suited for simple orchestration tasks and can be used within individual nodes of LangGraph for complex applications.

It is worth noting that the streaming capabilities discussed in our previous conversation related to LangChain more generally and applies to LCEL, as LCEL chains can be streamed, allowing for incremental output as the chain is executed, which is useful to reduce the time-to-first-token.


In [25]:
from langchain_core.runnables import RunnableSequence, RunnableLambda
from datetime import datetime


def runnable1(input):
    '''Runnable1'''
    print(f"Runnable1 - Current datetime: {datetime.now()}")
    return f"Output from runnable1: {input}"


def runnable2(input):
    '''Runnable2'''
    print(f"Runnable2 - Current datetime: {datetime.now()}")
    return f"Output from runnable2: {input}"

# runnable1 = RunnableLambda(runnable1)
# runnable2 = RunnableLambda(runnable2)

chain = RunnableSequence(runnable1, runnable2)
chain.invoke({"foo": "bar"})


Runnable1 - Current datetime: 2024-12-19 06:50:23.286430
Runnable2 - Current datetime: 2024-12-19 06:50:23.287213


"Output from runnable2: Output from runnable1: {'foo': 'bar'}"

In [26]:
from langchain_core.runnables import RunnableParallel
chain = RunnableParallel({
    "key1": runnable1,
    "key2": runnable2,
})
chain.invoke({"foo": "bar"})

Runnable1 - Current datetime: 2024-12-19 06:50:54.363559
Runnable2 - Current datetime: 2024-12-19 06:50:54.363851


{'key1': "Output from runnable1: {'foo': 'bar'}",
 'key2': "Output from runnable2: {'foo': 'bar'}"}

In [28]:
runnable1 = RunnableLambda(runnable1)
runnable2 = RunnableLambda(runnable2)

chain = runnable1.pipe(runnable2)

chain.invoke({"foo": "bar"})

Runnable1 - Current datetime: 2024-12-19 07:01:10.240102
Runnable2 - Current datetime: 2024-12-19 07:01:10.240217


"Output from runnable2: Output from runnable1: {'foo': 'bar'}"

In [29]:
chain = runnable1 | runnable2

chain.invoke({"foo": "bar"})

Runnable1 - Current datetime: 2024-12-19 07:01:53.332872
Runnable2 - Current datetime: 2024-12-19 07:01:53.332998


"Output from runnable2: Output from runnable1: {'foo': 'bar'}"

In [31]:
# Inside an LCEL expression, a dictionary is automatically converted to a RunnableParallel.

def runnable3(input):
    '''Runnable3'''
    print(f"Runnable3 - Current datetime: {datetime.now()}")
    return f"Output from runnable3: {input}"

runnable3 = RunnableLambda(runnable3)

mapping = {
    "key1": runnable1,
    "key2": runnable2,
}

chain = mapping | runnable3

print(chain.invoke({"foo": "bar"}))


Runnable1 - Current datetime: 2024-12-19 07:03:47.368899
Runnable2 - Current datetime: 2024-12-19 07:03:47.369949
Runnable3 - Current datetime: 2024-12-19 07:03:47.371110
Output from runnable3: {'key1': "Output from runnable1: {'foo': 'bar'}", 'key2': "Output from runnable2: {'foo': 'bar'}"}


In [33]:
chain = RunnableSequence(RunnableParallel(mapping), runnable3)

print(chain.invoke({"foo": "bar"}))

Runnable1 - Current datetime: 2024-12-19 07:04:58.797097
Runnable2 - Current datetime: 2024-12-19 07:04:58.797477
Runnable3 - Current datetime: 2024-12-19 07:04:58.797785
Output from runnable3: {'key1': "Output from runnable1: {'foo': 'bar'}", 'key2': "Output from runnable2: {'foo': 'bar'}"}
